In [2]:
import json
import pandas as pd
import sqlite3

In [2]:
with open('../projects/twitter/collected_tweets_2020-11-21-173426.txt', 'r') as file:
    tweets = file.readlines()

with open('tweet_example.json', 'w') as f:
    json.dump(json.loads(tweets[0]), f)

parsed_tweets = [json.loads(json.loads(_)) for _ in tweets]

In [3]:
def process_tweets(tweet):
    try:
        df = pd.DataFrame(tweet).reset_index(drop=True).iloc[:1]

        cols_to_del = ['quote_count', 'reply_count', 'retweet_count', 'favorite_count',
                   'favorited', 'retweeted', 'user', 'entities', ]
        df.drop(cols_to_del, axis=1, inplace=True)

        df['user_id'] = tweet['user']['id']
        df['user_id_str'] = tweet['user']['id_str']
        df['user_screen_name'] = tweet['user']['screen_name']
        df['user_location'] = tweet['user']['location']
        df['user_description'] = tweet['user']['description']
        df['user_protected'] = tweet['user']['protected']
        df['user_verified'] = tweet['user']['verified']
        df['user_followers_count'] = tweet['user']['followers_count']
        df['user_friends_count'] = tweet['user']['friends_count']
        df['user_created_at'] = tweet['user']['created_at']

        user_mentions = []
        for i in range(len(tweet['entities']['user_mentions'])):
            _tmp = tweet['entities']['user_mentions'][i].copy()
            _tmp.pop('indices', None)

            _df = pd.DataFrame(_tmp, index=[0])
            _df.rename(columns={
                'screen_name': 'entities_screen_name',
                'name': 'entities_name',
                'id': 'entities_id',
                'id_str': 'entities_id_str'
            }, inplace=True)
            user_mentions.append(_df)

        mentions = []
        for i in user_mentions:
            mentions.append(pd.concat([df.copy(), i], axis=1))

        result = pd.concat(mentions, ignore_index=True)
        return result
    except:
        return None

In [4]:
len(parsed_tweets)

3540

In [ ]:
treated_tweets = [process_tweets(tweet) for tweet in parsed_tweets]

In [ ]:
treated_tweets_not_null = [_tweet for _tweet in treated_tweets if _tweet is not None]

In [ ]:
df_final = pd.concat(treated_tweets_not_null, ignore_index=True, sort=False)

In [5]:
# df_final.to_csv('./backup.txt', sep=';', index=False)
df_final = pd.read_csv('./backup.txt', sep=';')

In [6]:
conn = sqlite3.connect('./twitter.db')
cur = conn.cursor()